In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch.autograd import Variable
from torch.nn import Linear, ReLU, MSELoss, Sequential, Conv1d, MaxPool1d, Module, LogSoftmax, BatchNorm1d, Dropout
from torch.optim import Adam, SGD

In [2]:
"""
Not the way to go must implement the Dataset class or no

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
"""
# Load the data into pandas dataframes, encode and split dynamic/static
# trainframe = pd.read_csv('train.csv')
# testframe = pd.read_csv('test.csv')
# trainframe['Activity'] = trainframe['Activity'].map(activity_encode)
# testframe['Activity'] = testframe['Activity'].map(activity_encode)
# dynamic = {}
# static = {}
#
# dynamic["trainframe"], static["trainframe"] = [x for _, x in trainframe.groupby(trainframe['Activity'] > 3)]
# dynamic["testframe"], static["testframe"] = [x for _, x in testframe.groupby(testframe['Activity'] > 3)]

'\nNot the way to go must implement the Dataset class or no\n\nhttps://pytorch.org/tutorials/beginner/data_loading_tutorial.html\n'

In [3]:
# get the validation set from the training set
#
# dynamic["trainset"], dynamic["validset"] = np.split(trainframe, [int(.2*len(trainframe)), int(.8*len(trainframe))])
# static["trainset"], static["validset"] = np.split(trainframe, [int(.2*len(trainframe)), int(.8*len(trainframe))])

# maybe dataframe to csv?
# implement dataset for this one
# or this
# https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/
# https://www.tutorialspoint.com/pytorch/pytorch_convolutional_neural_network.htm

In [4]:
# Data mm

In [5]:
activity_encode = {"WALKING":1,
       "WALKING_DOWNSTAIRS":2,
       "WALKING_UPSTAIRS":3,
       "LAYING":4,
       "SITTING":5,
       "STANDING":6
       }
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valid = pd.read_csv('valid.csv')
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [6]:
train['Activity'] = train['Activity'].map(activity_encode)
test['Activity'] = test['Activity'].map(activity_encode)
valid['Activity'] = valid['Activity'].map(activity_encode)

In [7]:
train_y = train['Activity'].values
train = pd.DataFrame(train.drop(['Activity','subject'],axis=1))
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [8]:
train_x = np.array(train)
#train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)

In [9]:
(train_x.shape, train_y.shape)#, (val_x.shape, val_y.shape)

((5879, 561), (5879,))

In [10]:
# [batch, channels, features]
# converting training images into torch format
train_x = train_x.reshape(5881, 1, 561)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

ValueError: cannot reshape array of size 3298119 into shape (5881,1,561)

In [ ]:
# converting validation images into torch format
val_x = val_x.reshape(1471, 1, 561)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

In [ ]:
# Models
# Understand the paper
class DynamicCNN(Module):
    def __init__(self):
        super(DynamicCNN, self).__init__()

        self.cnn_layers = Sequential(
            # a
            Conv1d(1, 100, kernel_size=tuple([3]), stride=tuple([1]), padding=tuple([1])),
            ReLU(inplace=True),
            # b
            MaxPool1d(kernel_size=3, stride=1, padding=1),
        )

        self.linear_layers = Sequential(
            Dropout(0.5),
            Linear(186*100, 3),
            ReLU(inplace=True),
            LogSoftmax(dim=1)
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
# defining the model
model = DynamicCNN()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0004)
# defining the loss function
criterion = MSELoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda().float()
        y_train = y_train.cuda().float()
        x_val = x_val.cuda().float()
        y_val = y_val.cuda().float()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

In [ ]:
# prediction for training set
with torch.no_grad():
    output = model(train_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

In [ ]:
# prediction for validation set
with torch.no_grad():
    output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)
